In [3]:
import pymongo
import pandas as pd
import matplotlib as plt
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures

market = futures.market
unit = futures.unit
position = futures.position

#加载数据
market = DataFrame(list(market.find()))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find()))

#大写字母
position['variety']=position['variety'].str.upper()

#删除/选取某行含有特殊数值的列
position=position.set_index('名次')

#选择需要显示的字段
data1=market[['date','variety','set_close']]

data2=unit[[ 'variety','unit']]
position=position[['date','variety','symbol','持买仓量期货公司','持买仓量', '持买仓量增减','持卖仓量期货公司','持卖仓量', '持卖仓量增减']]

#查询会员
members='国投安信期货'
data3=position[(position['持买仓量期货公司'] == members)]
#汇总合约
data3=data3[['date','variety','持买仓量期货公司','持买仓量']]
data3=data3.groupby(['date','variety','持买仓量期货公司'])[['持买仓量']].sum()
data4=position[(position['持卖仓量期货公司'] == members)]
data4=data4[['date','variety','持卖仓量期货公司','持卖仓量']]
data4=data4.groupby(['date','variety','持卖仓量期货公司'])[['持卖仓量']].sum()
print(data3.head())
print(data4.head())

#并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: members,axis=1)

#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['持买仓量']-x['持卖仓量'],axis=1)
#选择需要显示的字段
data5=data5[['会员简称','持买仓量','持卖仓量','净持仓']]
data5=data5.reset_index(['variety','date'])
print(data5.head())
netpostion=data5.set_index('date')

netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)

#合约价值
contractValue=pd.merge(data1,data2,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['set_close']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]
#值替换replace()
contractValue=contractValue.replace(['TA'],'PTA')
print(contractValue.head())
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

#净持仓价值
sz['净持仓价值']=sz.apply(lambda x: x['净持仓']*x['contractValue'],axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
sz['商品板块']=sz.sum(axis=1)
sz



                                持买仓量
date       variety 持买仓量期货公司         
2018-10-08 CF      国投安信期货    12236.0
           CY      国投安信期货        5.0
           JR      国投安信期货       45.0
           LR      国投安信期货      158.0
           OI      国投安信期货     7619.0
                                持卖仓量
date       variety 持卖仓量期货公司         
2018-10-08 CF      国投安信期货    10725.0
           JR      国投安信期货       51.0
           LR      国投安信期货       44.0
           MA      国投安信期货     6928.0
           OI      国投安信期货    42729.0
         date variety    会员简称     持买仓量     持卖仓量      净持仓
0  2018-10-08      CF  国投安信期货  12236.0  10725.0   1511.0
1  2018-10-08      CY  国投安信期货      5.0      0.0      5.0
2  2018-10-08      JR  国投安信期货     45.0     51.0     -6.0
3  2018-10-08      LR  国投安信期货    158.0     44.0    114.0
4  2018-10-08      OI  国投安信期货   7619.0  42729.0 -35110.0
         date variety  contractValue
0  2018-10-08      AG        52965.0
1  2018-10-08      AL        72395.0
2  2018-10-08      AP       1

,variety,AP,CF,CY,FG,JR,LR,MA,OI,PTA,RI,RM,SF,SM,SR,WH,ZC,商品板块
date,会员简称,,,,,,,,,,,,,,,,,
2018-10-08,国投安信期货,0,120441810,612200,0,-350040,6085320,-229593920,-2348859000,0,0,-734639700,-392466700,63697720,-93476500,13759120,10416700,-3584372990
2018-10-09,国投安信期货,0,87435915,0,0,-707760,11547440,-230309240,-2322073600,0,0,-747587700,-490414480,52759210,43616820,13872600,-459548000,-4041408795
2018-10-10,国投安信期货,0,43794125,-2213280,39462300,-590000,11852280,-246155040,-2437467760,0,0,-762728530,-552694940,91177560,14449680,13882880,-610689800,-4397920525
2018-10-11,国投安信期货,0,91241040,4269125,59016320,-587000,6561160,-259144600,-2330955000,0,141960,-767200920,-649966500,64719100,3239040,13935640,-623507600,-4388238235
2018-10-12,国投安信期货,0,43710830,1857375,74468640,-589400,3756360,-307251680,-2460921520,0,142320,-750796160,-682335530,112976920,46245500,14091000,-599898600,-4504543945
2018-10-15,国投安信期货,0,96171565,1365155,71567400,-588000,-1411800,-320163840,-2459244480,0,284520,-789862620,-706199820,131760350,27236300,14168000,-636675000,-4571592270
2018-10-16,国投安信期货,0,170247350,2472700,72929900,-822080,-7827100,-376721280,-2420282460,0,279960,-792354420,-659618250,77763500,-3689280,14036000,-634548800,-4558134260
2018-10-17,国投安信期货,0,156190995,-9179700,73133040,-884400,-7454760,-262774620,-2420940480,0,283920,-767200080,-665861310,70494240,6483960,13839360,-640706400,-4454576235
2018-10-18,国投安信期货,0,-26596295,1348930,71141280,-879300,-14585220,-19620370,-2349497640,0,281640,-753554560,-645613800,82000730,14078720,13726800,-596641500,-4224410585
